In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import time 
import numpy as np
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = 10, 10
from utilities import *
import math
import pandas as pd
from itertools import combinations

In [3]:
# read the data 
def process_inhval(*args):
    for val in args: 
        if val == 'NA':
            yield 0
        else:
            yield float(val)
readfilename = 'PI_DataSet_6_19_all.txt'
r = open(readfilename)
header = process_line(next(r))

In [4]:
# things to change 
#vectype = 'distvec'
vectype = 'countvec'
numinhs = 3
estimators = 2
testsize = 0.97

In [5]:
inh_start = header.index('FPV') 
inh_end = header.index('DRV')+1

vec_start = header.index('dist_vec_start') if vectype =='distvec' else header.index('count_vec_start')
vec_end = header.index('count_vec_start') if vectype =='distvec' else len(header)
vals = []
for line in r:
    if line!= '\n':
        line = process_line(line)
        token = list(process_inhval(*line[inh_start:inh_end])) + [float(x) for x in line[vec_start:vec_end]]
        vals.append(token)
r.close()
vals = np.array(vals)
ind_inh = {v:i for i,v in enumerate(header[inh_start:inh_end])}
def filter_inhs(data, ind_dict, *args):
    vec_length = 210
    names = sorted([(ind_dict[name],name) for i,name in enumerate(args)])
    inds = list(map(lambda x:x[0], names))
    names = list(map (lambda x:x[1], names))
    tmp = np.hstack((data[:,inds],data[:,8:]))
    df = pd.DataFrame(tmp, columns = ['c'+str(i) for i in range(len(tmp[0]))])
    query_string = ' & '.join(['c'+str(i)+">0" for i in range(len(inds))])
    df = df.query(query_string)
    for i,name in enumerate(names):
        df[names[i]] = np.where(df['c'+str(i)] > 3.0 , '1', '0')
    df['target'] = df[names].apply(lambda x:int(''.join(x),2),axis = 1)
    return df.iloc[:,len(names):(-len(names)-1)].values,df['target'].values.reshape(-1,1)

In [6]:
header = ["inh"+str(i+1) for i in range(numinhs)]+["rmse","score","time"]
results = pd.DataFrame(columns=header)
ind = 0 
inhs = ["ATV","DRV","FPV","IDV","LPV","NFV","SQV","TPV"]

In [7]:
combs = list(combinations(inhs,numinhs))
t0 = time.time()
for tup in combs:
    print(tup)
    X,y = filter_inhs(vals,ind_inh,*tup)
    max_ = X.max()
    X = X/max_
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testsize)
    model = RandomForestRegressor(n_estimators=estimators, bootstrap = False, max_features = 210, verbose = 1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = pow(mean_squared_error(y_test, y_pred),0.5)
    print("rmse:",printfour(rmse))
    score = model.score(X_test,y_test)
    print("score:", score)
    tt = printfour(time.time()-t0)
    print("time taken:",tt)
    print("______________")
    row = list(tup) + [rmse,score,tt]
    results.loc[ind] = row
    ind += 1

('ATV', 'DRV', 'FPV')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished


rmse: 0.4499
score: 0.9746642639551488
time taken: 4.1306
______________
('ATV', 'DRV', 'IDV')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s finished


rmse: 0.5226
score: 0.9649373298241751
time taken: 8.3096
______________
('ATV', 'DRV', 'LPV')


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s finished


score: 0.9757240633257906
time taken: 330.8961
______________
('NFV', 'SQV', 'TPV')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


rmse: 0.4029
score: 0.9679524684360367
time taken: 337.7028
______________


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s finished


In [8]:
filename = vectype + '_'+str(estimators)+'estimators_'+str(int(testsize*100))+'testsize_'+str(numinhs)+'inhibitors_randomforest_rmse_and_score.csv'
print(filename)

countvec_2estimators_97testsize_3inhibitors_randomforest_rmse_and_score.csv


In [9]:
results.round(4).to_csv(filename)